In [1]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
import json
from datetime import datetime as dt
from datetime import date
from statistics import mean
import math
import pickle
import time

In [2]:
# csv.field_size_limit(sys.maxsize)
# np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:20,.15f}'.format
# pd.options.display.max_rows = 10000
# pd.set_option('display.max_colwidth', -1)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
def calc_smooth_mean(df, by, on, m):
    mean = df[on].mean()
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']
    smooth = (counts * means + m * mean) / (counts + m)
    return df[by].map(smooth)

In [ ]:
df_file = pd.read_csv('savedcsv/17_homepage.csv')
df_file = df_file.drop(df_file[df_file['META__id'] == 6].index)
df_file.to_csv('datasets/df_file.csv', index=False)
df_file = pd.read_csv('datasets/df_file.csv')

In [5]:
# df_raw = reduce_mem_usage(pd.read_csv('savedcsv/17_homepage.csv'))
df_raw = pd.read_csv('savedcsv/17_homepage.csv')
df_raw['META__profitability'] = df_raw['profitability']
df_raw['META__year_avg_profitability'] = df_raw['year_avg_profitability']
df_raw = df_raw.drop(['profitability', 'year_avg_profitability', 'PROCESS__original_language'], axis=1)
drop_cols = [c for c in df_raw.columns if 'META' not in c and ('avg' in c or 'experience' in c or 'movies_before' in c)]
df_raw = df_raw.drop(drop_cols, axis=1)
df_raw = df_raw.drop(df_raw[df_raw['META__id'] == 6].index)
df_raw.to_csv('datasets/df_raw.csv', index=False)
df_raw = pd.read_csv('datasets/df_raw.csv')

In [6]:
df_raw = pd.read_csv('datasets/df_raw.csv')

In [15]:
df_raw.META__revenue.quantile([0.05, 0.95])

0.050000000000000       111,229.200000000011642
0.950000000000000   348,514,752.599999785423279
Name: META__revenue, dtype: float64

In [ ]:
num = df_raw[
    (df_raw.META__profitability > df_raw.META__profitability.quantile(0.01)) & 
    (df_raw.META__profitability < df_raw.META__profitability.quantile(0.99)) &
    (df_raw.META__year >= 2000) &
    (df_raw.META__year <= 2018) &
    (df_raw.budget >= 250000)
].shape[0]

print(num)
print(num/df_raw.shape[0])

In [ ]:
num = df_raw[
    (df_raw.META__revenue > df_raw.META__profitability.quantile(0.01)) & 
    (df_raw.META__profitability < df_raw.META__profitability.quantile(0.99)) &
    (df_raw.META__year >= 2000) &
    (df_raw.META__year <= 2018) &
    (df_raw.budget >= 250000)
].shape[0]

print(num)
print(num/df_raw.shape[0])

In [7]:
def get_real_columns(name_prefix, name_suffix, num_columns):
    return [f'{name_prefix}{j}{name_suffix}' for j in range(1, num_columns+1)]

def get_date_from_movie(movie):
    return date(movie['META__year'], movie['META__month'], movie['META__day'])


def get_info_df_movies_per_value_range_columns(df, columns):
    result = {}
    for i, row in df.iterrows():
        for col in columns:
            value = row[col]
            if not pd.isna(value):
                if value in list(result.keys()):
                    result[value]['movies'] = result[value]['movies'].append(df.iloc[[i]])
                else:
                    result[value] = {'movies': df.iloc[[i]]}
    return result

# def get_info_df_movies_per_value_range_columns(df, columns):
#     def rows_with_value(value):
#         return df[df.apply(lambda row: value in [row[column] for column in columns], axis=1)]

#     unique_values = list(set([movie[column] for i, movie in df.iterrows() for column in columns]))
#     return {value: rows_with_value(value) for value in unique_values}

def calculate_info_per_previous_movies_per_value(df, 
                                                 info_dict, 
                                                 calc_budget=False, 
                                                 calc_movies_before=False, 
                                                 calc_experience=False, 
                                                 calc_revenue=False,
                                                ):
    
    def get_average_profit(df):
        return (df['META__revenue'] - df['budget']).mean()
    
    def get_experience(df, row):
        return (row['META__year'] - df.META__year.min())
    
    def is_movie_before(movie, compare_date):
        return date(movie.META__year, movie.META__month, movie.META__day) < compare_date

    for value_name in info_dict:
        value = info_dict[value_name]
        value['avg_profit'] = {}
        if calc_budget:
            value['avg_budget'] = {}
        if calc_experience:
            value['experience'] = {}
        if calc_revenue:
            value['avg_revenue'] = {}
        if calc_movies_before:
            value['movies_before'] = {}

        movies = value['movies']
        for movie_index, movie in movies.iterrows():
            current_date = get_date_from_movie(movie)
            str_date = str(current_date)
            if calc_experience and str_date not in list(value['experience'].keys()):
                value['experience'][str_date] = get_experience(movies, movie)
            if str_date not in list(value['avg_profit'].keys()):
                movies_before = movies[movies.apply(lambda row: is_movie_before(row, current_date), axis=1)]
                if not movies_before.empty:
                    value['avg_profit'][str_date] = get_average_profit(movies_before)
                    if calc_movies_before:
                        value['movies_before'][str_date] = movies_before.shape[0]
                    if calc_revenue:
                        value['avg_revenue'][str_date] = movies_before['META__revenue'].mean()                        
                    if calc_budget:
                        value['avg_budget'][str_date] = movies_before['budget'].mean()
                    

def get_columns_from_info_dict(df, info_dict, real_columns, new_column_prefix='', nested_info=False):
    def get_first_key(d):
        return d[list(d.keys())[0]]
    
    new_columns_data = {}
    info_names = [key for key in get_first_key(info_dict).keys() if key != 'movies']
    print(info_names)
    for index, real_column in enumerate(real_columns):
        print(real_column)
        new_column_names = [f'{new_column_prefix}_{key}' if len(real_columns) == 1 else f'{new_column_prefix}_{index+1}_{key}' for key in info_names] 
        print(new_column_names)
            
        for new_column_name in new_column_names:
            new_columns_data[new_column_name] = []

        for movie_index, movie in df.iterrows():
            value = movie[real_column]
            str_date = str(get_date_from_movie(movie))
            if pd.isna(value):
                for col in new_column_names:
                    new_columns_data[col].append(np.nan)
            else:
                for name_index, new_column_name in enumerate(new_column_names):
                    year_info = info_dict[value][info_names[name_index]].get(str_date, np.nan)
                    new_columns_data[new_column_name].append(year_info)
    return new_columns_data

In [8]:
def add_features(df, 
                 columns,
                 new_column_prefix,
                 calc_movies_before=False, 
                 calc_budget=False, 
                 calc_experience=False,
                 calc_revenue=False):
    print('obtaine movies')
    
    t = time.process_time()
    data_info = get_info_df_movies_per_value_range_columns(df, columns)
    print(f'time: {time.process_time() - t}')
    
    data_info_copy = data_info.copy()
    print('calculate info')
    t = time.process_time()
    calculate_info_per_previous_movies_per_value(df, 
                                                 data_info_copy, 
                                                 calc_movies_before=calc_movies_before, 
                                                 calc_budget=calc_budget, 
                                                 calc_experience=calc_experience,
                                                 calc_revenue=calc_revenue)
    print(f'time: {time.process_time() - t}')
    
    print('get columns')
    t = time.process_time()
    feature_columns = get_columns_from_info_dict(df, data_info_copy, columns, new_column_prefix=new_column_prefix)
    print(f'time: {time.process_time() - t}')
    for column in feature_columns.keys():
        df[column] = feature_columns[column]
    return df


def add_companies_features(df):
    columns = get_real_columns('META__production_company_', '', 3)
    return add_features(df, columns, 'production_company', calc_revenue=True)


def add_cast_features(df):
    columns = get_real_columns('META__cast_', '_name', 8)
    return add_features(df, columns, 'cast', calc_movies_before=True, calc_revenue=True, calc_experience=True)


def add_crew_features(df):
    for column in [c for c in list(df.columns) if 'META__crew' in c]:
        df = add_features(df, [column], column.replace('META__', ''), calc_movies_before=True, calc_revenue=True)
    return df


def add_collection_features(df):
    return add_features(df, ['META__collection_name'], 'collection', calc_revenue=True)


def get_avg_cast_info(df):
    def get_average_cast_movies(row, columns):
        avgs = [row[col] for col in columns if not pd.isna(row[col])]
        return mean(avgs) if len(avgs) else np.nan
    
    def get_cast_average(df, column_name):
        columns = [col for col in df.columns if 'cast_' in col and column_name in col and not 'cast_avg' in col]
        return [get_average_cast_movies(row, columns) for i, row in df.iterrows()]
    
    for column in ['revenue', 'profit', 'experience', 'movies_before']:
        df[f'cast_avg_{column}'] = get_cast_average(df, column)
    return df


def prepare_df(df_original):
    overall_time = time.process_time()
    df = df_original.copy()
    df = add_companies_features(df)
    df = add_cast_features(df)
    df = add_crew_features(df)
    df = add_collection_features(df)
    df = get_avg_cast_info(df)
    df['year_avg_revenue'] = calc_smooth_mean(df, by='META__year', on='META__revenue', m=5)
    print(f'overall time: {time.process_time() - overall_time}')
    return df
    

In [ ]:
df_all = prepare_df(df_raw)

In [17]:
raw_datasets = {
#     'us': df_raw[
#         df_raw.country__us == 1].copy(),
#     'gb': df_raw[
#         df_raw.country__gb == 1].copy(),
#     'years2000-2018': df_raw[
#         (df_raw.META__year >= 2000) &
#         (df_raw.META__year <= 2018)].copy(),
#     'years1970-1999': df_raw[
#         (df_raw.META__year >= 1970) &
#         (df_raw.META__year <= 1999)].copy(),
#     'profitability_positive': df_raw[
#         df_raw.META__profitability > 0].copy(),
#     'profitability_negative': df_raw[
#         df_raw.META__profitability < 0].copy(),
#     'budget_start_1percent': df_raw[
#         df_raw.budget >= 8875].copy(),
#     'budget_start_5percent': df_raw[
#         df_raw.budget >= 250000].copy(),
#     'profitability_1percentile': df_raw[
#         (df_raw.META__profitability >= -138) &
#         (df_raw.META__profitability <= 65)].copy(),
#     'profitability_5percentile': df_raw[
#         (df_raw.META__profitability >= -14.9) &
#         (df_raw.META__profitability <= 13.9)].copy(),
#     'profitability_10percentile': df_raw[
#         (df_raw.META__profitability >= -4.42) &
#         (df_raw.META__profitability <= 7.4)].copy(),
#     'profitability_1percentile_years2000-2018': df_raw[
#         (df_raw.META__profitability >= -138) &
#         (df_raw.META__profitability <= 65) &
#         (df_raw.META__year >= 2000) &
#         (df_raw.META__year <= 2018)].copy(),
#     'profitability_1percentile_years2000-2018_budget_start_1percent': df_raw[
#         (df_raw.META__profitability >= -138) &
#         (df_raw.META__profitability <= 65) &
#         (df_raw.META__year >= 2000) &
#         (df_raw.META__year <= 2018) &
#         (df_raw.budget >= 8875)].copy(),
#     'profitability_1percentile_years2000-2018_budget_start_5percent': df_raw[
#         (df_raw.META__profitability >= -138) &
#         (df_raw.META__profitability <= 65) &
#         (df_raw.META__year >= 2000) &
#         (df_raw.META__year <= 2018) &
#         (df_raw.budget >= 250000)].copy(),
    'revenue_1percentile': df_raw[
        (df_raw.META__revenue >= 10000) &
        (df_raw.META__revenue <= 854060072)].copy(),
    'revenue_5percentile': df_raw[
        (df_raw.META__revenue >= 111229) &
        (df_raw.META__revenue <= 348514752)].copy(),
}

In [18]:
for d in raw_datasets.keys():
    print(raw_datasets[d].shape[0])
for d in raw_datasets.keys():
    print(raw_datasets[d].shape[0]/df_raw.shape[0]*100)

7354
6743
98.1449352729214
89.99065794741759


In [19]:
for dataset in raw_datasets.keys():
    print('------------------------------------------------------')
    print(dataset)
    print('-------------------------------------------------------')
    raw_filename = f'datasets/dataset_{dataset}_raw.csv'
    filename = f'datasets/dataset_{dataset}.csv'
    raw_datasets[dataset].to_csv(raw_filename, index=False)
    raw_dataset = pd.read_csv(raw_filename)
    prepared = prepare_df(raw_dataset)
    prepared.to_csv(filename, index=False)

------------------------------------------------------
revenue_1percentile
-------------------------------------------------------
obtaine movies
time: 31.96551084800012
calculate info
time: 46.38811088300008
get columns
['avg_profit', 'avg_revenue']
META__production_company_1
['production_company_1_avg_profit', 'production_company_1_avg_revenue']
META__production_company_2
['production_company_2_avg_profit', 'production_company_2_avg_revenue']
META__production_company_3
['production_company_3_avg_profit', 'production_company_3_avg_revenue']
time: 1.9123951279998437
obtaine movies
time: 104.5923945350005
calculate info
time: 78.1556760100002
get columns
['avg_profit', 'experience', 'avg_revenue', 'movies_before']
META__cast_1_name
['cast_1_avg_profit', 'cast_1_experience', 'cast_1_avg_revenue', 'cast_1_movies_before']
META__cast_2_name
['cast_2_avg_profit', 'cast_2_experience', 'cast_2_avg_revenue', 'cast_2_movies_before']
META__cast_3_name
['cast_3_avg_profit', 'cast_3_experience', 'c

time: 0.6465045790000659
obtaine movies
time: 3.0112058980002985
calculate info
time: 2.04953417500019
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__property_master
['crew__art__property_master_avg_profit', 'crew__art__property_master_avg_revenue', 'crew__art__property_master_movies_before']
time: 0.652182638999875
obtaine movies
time: 6.709585949000029
calculate info
time: 4.6812005909996515
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__set_decoration
['crew__art__set_decoration_avg_profit', 'crew__art__set_decoration_avg_revenue', 'crew__art__set_decoration_movies_before']
time: 0.6560054450001189
obtaine movies
time: 2.7120818550001786
calculate info
time: 1.998631242000556
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__visualeffects__visual_effects_supervisor
['crew__visualeffects__visual_effects_supervisor_avg_profit', 'crew__visualeffects__visual_effects_supervisor_avg_revenue', 'crew__visualeffec

time: 7.6015660809998735
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__editing__editor__1
['crew__editing__editor__1_avg_profit', 'crew__editing__editor__1_avg_revenue', 'crew__editing__editor__1_movies_before']
time: 0.5983912980000241
obtaine movies
time: 8.076420693999353
calculate info
time: 5.811330103999353
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__costume__costume_designer
['crew__costume__costume_designer_avg_profit', 'crew__costume__costume_designer_avg_revenue', 'crew__costume__costume_designer_movies_before']
time: 0.6159680970004047
obtaine movies
time: 2.2119242019998637
calculate info
time: 1.5732701349998024
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__costume__costume_supervisor
['crew__costume__costume_supervisor_avg_profit', 'crew__costume__costume_supervisor_avg_revenue', 'crew__costume__costume_supervisor_movies_before']
time: 0.6103592039999057
obtaine movies
time: 3.039570779999849
cal